In [6]:
print("OM NAMOH BHAGWATE VASUDEVAYE NAMAH")

OM NAMOH BHAGWATE VASUDEVAYE NAMAH


In [7]:
import os
os.chdir("../")

In [8]:
%pwd

'/Users/yash_new/Desktop/AI-Medical-Chatbot'

In [9]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
#  Extract text from PDF files
def load_pdf_files(data):
    loader=DirectoryLoader(
        data,
        glob="*.pdf",loader_cls=PyPDFLoader)
    
    documents=loader.load()
    return documents

In [11]:
extracted_data = load_pdf_files("research/data")

In [ ]:
extracted_data

In [13]:
len(extracted_data)

759

In [14]:
# cleans & minimizes documents before embedding or storage

from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []

    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )

    return minimal_docs


In [15]:
minimal_docs = filter_to_minimal_docs(extracted_data)
len(minimal_docs)

759

In [3]:
# split the documents to the smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
    )

    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk



In [16]:
texts_chunk = text_split(minimal_docs)
len(texts_chunk)

4110

In [17]:
texts_chunk

[Document(metadata={'source': 'research/data/The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND copy.pdf'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION'),
 Document(metadata={'source': 'research/data/The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND copy.pdf'}, page_content='The G ALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nC-F\n2'),
 Document(metadata={'source': 'research/data/The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND copy.pdf'}, page_content='STAFF\nJacqueline L. Longe, Project Editor\nDeirdre S. Blanchfield, Associate Editor\nChristine B. Jeryan, Managing Editor\nDonna Olendorf, Senior Editor\nStacey Blachford, Associate Editor\nKate Kretschmann, Melissa C. McDade, Ryan\nThomason, Assistant Editors\nMark Springer, Technical Specialist\nAndrea Lopeman, Programmer/Analyst\nBarbara J. Yarrow,Manager, Imaging and Multimedia\nContent\nRobyn V . Young,Project Manager, Imaging and\nMultimedia Content\nDean

In [19]:
from langchain.embeddings import HuggingFaceEmbeddings
import torch

def download_embeddings():
    """
    Download and return HuggingFace embeddings model
    """
    model_name = "BAAI/bge-small-en-v1.5"

    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
    )

    return embeddings

embedding=download_embeddings()


/var/folders/zn/2rght4q50893h63h7nxfbkjm0000gn/T/ipykernel_78532/998734459.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [20]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='BAAI/bge-small-en-v1.5', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [21]:
vector=embedding.embed_query("Hello World")
vector

[0.015196125954389572,
 -0.022570684552192688,
 0.008547079749405384,
 -0.07417061179876328,
 0.003836425021290779,
 0.0027135368436574936,
 -0.0312679260969162,
 0.04463404789566994,
 0.04405522346496582,
 -0.007871137000620365,
 -0.025200802832841873,
 -0.03336653858423233,
 0.014427891001105309,
 0.04653818532824516,
 0.008555077016353607,
 -0.016145767644047737,
 0.007405787706375122,
 -0.019012434408068657,
 -0.11472620069980621,
 -0.018157636746764183,
 0.12635931372642517,
 0.02970287948846817,
 0.025281008332967758,
 -0.034217871725559235,
 -0.04099969193339348,
 0.006617291364818811,
 0.010270638391375542,
 0.02236226014792919,
 0.0044363392516970634,
 -0.1273096203804016,
 -0.0161492470651865,
 -0.020380135625600815,
 0.04721209034323692,
 0.011579901911318302,
 0.0681871771812439,
 0.007298652082681656,
 -0.017853032797574997,
 0.04078217223286629,
 -0.01026942953467369,
 0.0237570907920599,
 0.010602897964417934,
 -0.02858438529074192,
 0.008159657940268517,
 -0.01518052164

In [ ]:
print("Vector length:",len(vector))

In [32]:
# this will load your env file
from dotenv import load_dotenv
import os
load_dotenv()

True

In [30]:
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [31]:
from pinecone import Pinecone
pinecone_api_key=PINECONE_API_KEY
pc=Pinecone(api_key=pinecone_api_key)

In [25]:
pc

In [35]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec

# load env
load_dotenv()

# init pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

index_name = "medical-chatbot"

# list existing indexes
existing_indexes = [i["name"] for i in pc.list_indexes()]

# create index if not exists
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",   
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

# connect to index
index = pc.Index(index_name)

print(" Pinecone index ready:", index_name)


 Pinecone index ready: medical-chatbot


In [36]:
# to store vector

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)


In [ ]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore
#Embed each chunk and upsert the embeddings into the Pinecone index.
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

ADD MORE DATA TO THE EXISTING PINECONE INDEX

In [37]:
dswidth=Document(
    page_content="Yash Singh is a 3rd year Student of B.Tech and he is a lonely person,want to be his a good friend??",
    metadata={"Source":"Personal"}
)

In [38]:
docsearch.add_documents(documents=[dswidth])

['4da31fb4-db91-4602-8557-c8da8a4686cd']